In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from arithmetic.dataset import ModArithmeticCpDataset, get_mod_arithmetic_cp_dataloaders, ArithmeticDatasetConfig
from arithmetic.vis import visualize_data

data_config = ArithmeticDatasetConfig(
  p=5,
  noise_frac=0.5
)

train_dataloader, _, _ = get_mod_arithmetic_cp_dataloaders(data_config, batch_size=1, val_frac=0.5)

visualize_data(train_dataloader)

AttributeError: 'ModArithmeticCpDataset' object has no attribute 'data'

In [ ]:
import torch.nn as nn
from models.other import MLP

model = MLP(
  dims=[2, 128, 128, data_config.p]
)


In [65]:
torch.arange(p, dtype=torch.int32)

tensor([0, 1, 2, 3, 4], dtype=torch.int32)